In [ ]:
import numpy as np
import cv2
import math 
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from matplotlib import colors
from scipy.signal import argrelextrema
import pandas as pd
import csv
import os
from scipy.ndimage import gaussian_filter1d
import re

In [ ]:
extension_factor = 1.1
step_size = 2
max_width = 2048
overlap_thresh = 0.3
weight_epithelium_coverage = 0.8
weight_background_pixels = 0.2
sigma_factor = 20

## Base Patching Algorithm (dated 04/27)

In [ ]:
def estimate_curvature_complexity(points):
    """
    Estimate complexity of a contour based on curvature variations.
    """

    if len(points)<5:
        return 1
    
    diffs=np.gradient(points, axis=0)
    curvature=np.linalg.norm(np.gradient(diffs,axis=0),axis=1)

    # Count number of local maxima in curvature (sharp points)
    peaks=argrelextrema(curvature, np.greater)[0]
    
    #Return peaks/points for rough approx of sharp changes
    return len(peaks)/len(points)

In [ ]:

def find_external_contours(binary_image):
    '''find all ext`ernal contours of an image'''

    contours,hierarchy=cv2.findContours(binary_image,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    #This can be used to alter the order contour points are processed in while maintaining order
        #Think [1,2,3,4,5]->[4,5,1,2,3]
        #Set to 0 for no effect, check np documentation for more info
    return [np.roll(contour,0,axis=0) for contour in contours if len(contour)>1]


def gaussian_normals(contours):
    #Initialize list for final normals
    all_normals=[]
    smooth=[]

    #Loop through all contours
    for contour in contours:
        #Reshape contour into point pairings
        points=contour.reshape(-1,2).astype(np.float32)
        
        #Define sigma paramter(std for Gaussian Kernel from contour length)
        #Higher value of sigma means more aggressive smoothing
        #sigma,smoothed=estimate_sigma(points,1,1e-5,60)
        sigma=max(len(contour)*estimate_curvature_complexity(points)//sigma_factor,1)
        smoothed=gaussian_filter1d(points,sigma,axis=0, truncate = 100)
        print(f'Sigma of : {sigma}')
        #Apply Gaussian smoothing to X and Y values, attempting to remove noise from the contour
        #smoothed=np.apply_along_axis(lambda x: gaussian_filter1d(x,sigma,mode='reflect'),axis=0,arr=points)
        #Calculate gradients of X and Y for use then finding the tangent
        gradients=np.gradient(smoothed,axis=0)
        #Find vector norms of tangent vectors
        all_norms=np.linalg.norm(gradients,axis=1,keepdims=True)
        #Divide tangents by norms with numerical stability
        tangent_unit=gradients/(all_norms+1e-8)
        #Initialize normals
        normals=np.zeros_like(tangent_unit)
        
        normals = np.column_stack((-tangent_unit[:, 1], tangent_unit[:, 0]))
        all_normals.append(normals)
        smooth.append(smoothed.reshape(-1,1,2).astype(np.int32))

    return all_normals,smooth


def calculate_patch_corners(point, normal, binary_image, extension_factor=1.0, step_size=4, max_length=2048):
    """
    Calculate the corners of a square patch using epithelium width along the normal.

    Args:
        point (np.ndarray): The point on the contour.
        normal (np.ndarray): The normal vector at the point.
        binary_image (np.ndarray): Binary mask of the tissue.
        extension_factor (float): Factor to slightly enlarge the patch (e.g., 1.1 for 10% increase).
        step_size (int): Step size for checking tissue boundaries along the normal.
        max_length (int): Maximum distance to search for epithelium width.

    Returns:
        np.ndarray: The four corners of the square patch.
        int: The computed patch height.
    """
    height, width = binary_image.shape
    point = point.astype(int)

    # generating forward and backward points along the normal
    steps = np.arange(0, max_length, step_size)[:, None]
    points = np.clip(point - steps * normal, 0, [width - 1, height - 1]).astype(int)

    # extracting pixel values along the normal path
    values = binary_image[points[:, 1], points[:, 0]]

    # finding the first background pixel (where value == 0)
    stop = np.where(values == 0)[0]

    # computing distances
    width = stop[0] * step_size if stop.size > 0 else max_length

    # computing total epithelium width and adjust for stroma extension
    extended_length = int(width * extension_factor)
    half_size = extended_length // 2 

    # computing tangent vector (perpendicular to normal)
    tangent = np.array([-normal[1], normal[0]])

    # defining square corners (same logic as `calculate_square_corners`)
    corners = np.array([
        point + (tangent * half_size),
        point - (tangent * half_size),
        point - (tangent * half_size) - (normal * extended_length),
        point + (tangent * half_size) - (normal * extended_length)
    ])

    return corners, extended_length


def calculate_square_overlap(square1,square2):
    '''Calculate overlap ratio between squares'''

    square1=Polygon(square1)
    square2=Polygon(square2)

    if not square1.intersects(square2):
        return 0
    
    intersection_area=square1.intersection(square2).area
    smallest_area=min(square1.area,square2.area)
    return intersection_area/(smallest_area+1e-8)



def create_smarter_squares(contours,normals,binary_image,overlap_thresh):
    '''Create squares that meet overlap thresh with last square'''
    return_squares=[]
    return_patch_length = []
    
    #Loop over all contours and get corresponding normals to point
    for contour, contour_normals in zip(contours,normals):
        points=contour.reshape(-1,2)
        squares=[]
        patch_length = []
        last_square=None

        #Go over points, evaluating if next square's corner falls outside of range of last square
        for i,point in enumerate(points):

            corners,length=calculate_patch_corners(point,contour_normals[i],binary_image)
            overlap_thresh_size_fn = min(0.8,overlap_thresh + max((length - 100)*1.5*overlap_thresh/900,0))
            
            #Prevent poorly sized squares
            #Finding rough area approximation in terms of patch length
            side_length = np.sqrt(cv2.contourArea(contour))
            #Approximately: Want squares less than 50% of contour area, and minimum of 2% area
            if length<side_length*0.02:
                continue
            #Check to see if first square and check agaisnt last square to prevent long loop
            elif last_square is not None and calculate_square_overlap(last_square,corners)>overlap_thresh_size_fn:
                if Polygon(last_square).area>Polygon(corners).area and calculate_square_overlap(last_square,corners)>.98:
                    squares[-1]=corners
                    last_square=corners
                    continue
                continue

            #Check agasint all squares, prevents issues of larger patches covering thin sections of epithelium next to large ones 
            elif any(calculate_square_overlap(corners, square)>overlap_thresh_size_fn for square in squares)==True:
                continue
        
            squares.append(corners)
            patch_length.append(length)
            last_square=corners
        return_squares.append(squares)
        return_patch_length.append(patch_length)
    return return_squares, return_patch_length

def draw_squares(image,squares_list,color=[255,0,0],thickness=20):
    '''Draw squares on image'''

    result=image.copy()

    for squares in squares_list:
        for square in squares:
            points=square.astype(np.int32)
            cv2.polylines(result,[points],isClosed=True,color=color,thickness=thickness)
        
    return result

# Function to draw patches
def draw_patches(image, patch_list, color = [255,0,0], thickness = 20):
    patches_on_slice = image.copy()
    for squares in patch_list:
        for square in squares:
            points=square.astype(np.int32)
            cv2.polylines(patches_on_slice,[points],isClosed=True,color=color,thickness=thickness)
    return patches_on_slice



def process_contours(image,binary_image,overlap_thresh):
    """
    Main function to process multiple contours and generate aligned squares along them.
    
    Args:
        image (np.ndarray): Original image
        binary_image (np.ndarray): Binary mask of the shapes
        overlap_thresh (float): Maximum allowed overlap between squares        
    Returns:
        tuple: (processed image, contours, squares_list)
    """
    contours=find_external_contours(binary_image)
    if not contours:
        return TypeError('No contours found')
    
    #Condition check for square creation
    #normals=calculate_contour_normals(contours,smoothing_size)
    normals,smooth=gaussian_normals(contours)
    #normals,smooth=fourier_normals(contours,0.999)
    final_squares_list, patch_length_list = create_smarter_squares(contours,normals,binary_image,overlap_thresh=overlap_thresh)
    tissue_slice = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    
    result = draw_squares(tissue_slice,final_squares_list)
    #Visualzie smoothed contour

    try:
        cv2.drawContours(result,smooth,-1,(0,255,0),30)
    except:
        print('No smoothed contour to show')
    
    return result, contours, final_squares_list, normals, patch_length_list

# Carried over functions

def empty_folder():
    # delete everything out of directory
    output_folder = 'testing_slices/patch_outputs'
    if not os.path.exists(output_folder):  # Check if folder exists
        os.makedirs(output_folder)  # Create folder if it doesn't exist
    for file in os.listdir(output_folder):
        file_path = os.path.join(output_folder, file)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            os.rmdir(file_path)

# Export patched slice into a new folder 
def export_patched_slice(patched_slice_name, selected_patches_on_slice):

    output_folder = os.path.join('testing_slices', 'patch_images')
    if not os.path.exists(output_folder):  # Check if folder exists
        os.makedirs(output_folder)  # Create folder if it doesn't exist

    output_path = os.path.join(output_folder, os.path.basename(patched_slice_name))
    cv2.imwrite(output_path, cv2.cvtColor(selected_patches_on_slice, cv2.COLOR_RGB2BGR))

# Calculate performance metrics
def calculate_metrics(epithelium_mask_2D, all_selected_patches):

    num_patches = len(np.vstack(all_selected_patches))

    total_epithelium_pixels = (epithelium_mask_2D > 0).sum()
    patch_mask = np.zeros_like(epithelium_mask_2D, dtype=np.uint8)

    #For all patches in all contours, use them to create a mask
    for contour_patches in all_selected_patches:
        for patch in contour_patches:
            patch = np.array(patch,dtype = np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(patch_mask, [patch], 255)

    number_of_patch_mask_pixels = np.count_nonzero(patch_mask > 0)

    covered_epithelium_pixels = np.count_nonzero((patch_mask > 0) & (epithelium_mask_2D > 0))
    covered_background_pixels = np.count_nonzero((patch_mask > 0) & (epithelium_mask_2D == 0))

    total_epithelium_coverage = (covered_epithelium_pixels / total_epithelium_pixels*100)
    background_pixel_percent = (covered_background_pixels / number_of_patch_mask_pixels*100)
    score = weight_epithelium_coverage*total_epithelium_coverage + weight_background_pixels*(100 - background_pixel_percent)

    # Exporting performance metrics and parameter values
    metrics = pd.DataFrame({'Metric':['num_patches','total_epithelium_coverage', 'background_pixel_percent',
                        'score', 'extension_factor', 'step_size',
                        'max_width', 'overlap_threshold'], 
                'Value':[num_patches, total_epithelium_coverage, background_pixel_percent, score, 
                        extension_factor, step_size, max_width, overlap_thresh]})

    return metrics

def find_edges(all_selected_patches):
    all_points = np.vstack(all_selected_patches[0])

    edges = {'min_x': np.min(all_points[:, 0]),
             'max_x': np.max(all_points[:, 0]),
             'min_y': np.min(all_points[:, 1]),
             'max_y': np.max(all_points[:, 1])}

    return edges

def add_padding(image, edges):
        
    height, width, channels = image.shape

    if edges['min_x'] < 0:
        left = math.ceil(np.abs(edges['min_x']))
    else: left = 0

    if edges['max_x'] > width:
        right = math.ceil(edges['max_x'] - width)
    else: right = 0

    if edges['min_y'] < 0:
        top = math.ceil(np.abs(edges['min_y']))
    else: top = 0

    if edges['max_y'] > height:
        bottom = math.ceil(edges['max_y'] - height)
    else: bottom = 0

    new_width = width + right + left
    new_height = height + top + bottom
    

    # compute the padding color using the mode pixel value
    threshold = 150
    mask = np.all(image>threshold, axis = -1)
    if np.any(mask):
        bright_pixels = image[mask]
        if np.issubdtype(image.dtype,np.integer):
            dims = (np.iinfo(image.dtype).max + 1,) * channels
            combined = np.ravel_multi_index(bright_pixels.T,dims)
            unique_vals, counts = np.unique(combined, return_counts = True)
            mode_val = unique_vals[np.argmax(counts)]
            mode_color = np.array(np.unravel_index(mode_val, dims)).astype(image.dtype)
            pad_color = mode_color.flatten()
        else:
            pixels, counts = np.unique(bright_pixels, axis = 0, return_counts = True)
            pad_color = pixels[np.argmax(counts)]
    else:
        pad_color = np.array([threshold] * channels , dtype = image.dtype)


    padded_image = np.ones((new_height, new_width, channels), dtype=image.dtype) * pad_color

    # Insert the original image into the array of updated dimensions
    padded_image[top:top + height, left:left + width] = image


    return padded_image, right, left, top, bottom

# Shift patches by padding amount
def shift_patches(padded_image, all_selected_patches, left, top):

    all_points = np.vstack(all_selected_patches)

    shift_x = left
    shift_y = top
    shift_array = np.array([shift_x, shift_y])
    shifted_points = all_points + shift_array
    shifted_points = np.vstack(shifted_points)
    shifted_points = np.split(shifted_points, len(shifted_points) // 4)
    shifted_points = [shifted_points]   

    shifted_patches_on_slice = draw_patches(padded_image, shifted_points)

    return shifted_patches_on_slice, shifted_points

# edited to 
def show_shifted_patches(image, padded_image, all_selected_patches, shifted_patches_on_slice, binary_image):
    # Convert grayscale mask to RGB for overlaying patches
    mask_rgb = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB)

    # Plot the original image and the patched mask side by side
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    result = draw_patches(image,all_selected_patches)
    mask = draw_patches(mask_rgb,all_selected_patches)
    #mask = mask_rgb
    axes[0].imshow(result)
    axes[0].set_title("Original Image")
    axes[0].axis("off")
    axes[1].imshow(mask)
    axes[1].set_title("Grayscale Mask with Patches")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()
    result = cv2.hconcat([result,mask])
    return result

def extract_square_images(image, points):
    
    patch_images = []
    square_list = np.vstack(points)
    coords = []

    for square in square_list:

        # create an empty mask with white inside just the square's region
        square = square.astype(np.int32)
        square.reshape((-1,1,2)) # bc cv2.fillpoly expects points to be in shape (N,1,2)
        mask = np.zeros(image.shape[:2],dtype = np.uint8)
        cv2.fillPoly(mask, [square], 255)

        # Apply the mask to the image
        masked_image = cv2.bitwise_and(image, image, mask = mask)

        # uncomment to see placement of patch within the entire slice image 
        # plt.imshow(masked_image)
        # plt.show()

        # Crop the image around the rotated square (will have black space due to rotation)
        x_min = np.min(square[:, 0])
        x_max = np.max(square[:, 0])
        y_min = np.min(square[:, 1])
        y_max = np.max(square[:, 1])
        cropped_image = masked_image[y_min:y_max, x_min:x_max]
        
        coords.append((x_min, y_min, x_max, y_max))

        # rotate the cropped image to be parallel to the xy axis
        # calculate the angle the patch needs to be rotated by
        point1 = square[0]  # top-left corner
        point2 = square[1]  # top-right corner
        vector = point2 - point1
        angle_degrees = np.arctan2(vector[1], vector[0]) * (180.0 / np.pi) - 180

        # Get the center of the cropped image
        center = (int((x_max - x_min)) // 2, int((y_max - y_min)) // 2)

        # Create the rotation matrix to rotate around the center
        rotation_matrix = cv2.getRotationMatrix2D(center, angle_degrees, 1.0) # 1.0 means image wont be scaled

        # Rotate the cropped image to make it parallel to xy axis
        if cropped_image.shape[0] > 0 and cropped_image.shape[1] > 0:
            rotated_patch = cv2.warpAffine(cropped_image, 
                                        rotation_matrix,
                                        (cropped_image.shape[1], cropped_image.shape[0]))  # output image size
        else:
            print("Cropped image is empty, skipping rotation.")

        # remove the extra black pixels around the edges
        gray = cv2.cvtColor(rotated_patch, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x, y, w, h = cv2.boundingRect(contours[0])
        cropped_img = rotated_patch[y:y+h, x:x+w]

        patch_images.append(cropped_img)
    
    return patch_images, coords

def save_patches(patch_images, slice_name):
    # Ensure output directory exists
    output_folder = os.path.join("testing_slices","patch_outputs")
    if not os.path.exists(output_folder):  # Check if folder exists
        os.makedirs(output_folder)  # Create folder if it doesn't exist
    # Clean file name
    file_name = os.path.basename(slice_name).replace('mask', '').replace('.png', '')
    
    # Save patches
    for i, patch in enumerate(patch_images):
        if isinstance(patch, np.ndarray):
            output_path = os.path.join(output_folder, f"{file_name}_patch{i}.png")
            cv2.imwrite(output_path, patch)

def patching_export(binary, slice_name, slice_details, stain_images):
    " processes tissue slice to create and export patches"
    
    result, contours, final_squares, normals, patch_length = process_contours(slice_details, binary, overlap_thresh)    

    # find edges and add padding
    edges = find_edges(final_squares)
    tissue_slice = cv2.cvtColor(cv2.imread(slice_details), cv2.COLOR_BGR2RGB)
    padded_image, right, left, top, bottom = add_padding(tissue_slice, edges)
    _, shifted_points = shift_patches(padded_image, final_squares, left, top)
    shifted_points = [tuple(pt[0]) if isinstance(pt, np.ndarray) and pt.ndim == 2 else tuple(pt) for pt in shifted_points]

    # export patched slice
    patched_slice_name = slice_details.replace('testing_slices/', 'patched_')
    export_patched_slice(patched_slice_name.replace('.tif', '.png'), result)

    # extract square patch images

    patches, coords_ = extract_square_images(padded_image, shifted_points)

    # Get patches across stains
    
    patches_across_stains, coords_across_stains = get_patches_across_stains(
        patches,
        coords_,
        stain_images[0],
        stain_images[1],
        stain_images[2]
    )

    # save patches to output folder
    save_patches(patches_across_stains, slice_name)

    return patches_across_stains, patch_length


## Patching Across Stains (Adapted from Aryaman's Code)

In [ ]:
def normalize_coords(ref_image, new_image, coords):
    ref_y, ref_x = ref_image.shape[:2]
    new_y, new_x = new_image.shape[:2]

    xmin_norm = int(coords[0]/ref_x * new_x)
    ymin_norm = int(coords[1]/ref_y * new_y)

    xmax_norm = int(coords[2]/ref_x * new_x)
    ymax_norm = int(coords[3]/ref_y * new_y)

    return xmin_norm, ymin_norm, xmax_norm, ymax_norm

def get_patches_across_stains(patches, coords, image1, image2, image3):
    # Check the same region across all stains
    patches_across_stains = []
    coords_across_stains = []
    print("Patches entering get_patches_across_stains:", len(patches))
    for i, patch in enumerate(patches):
        min_x, min_y, max_x, max_y = coords[i]

        x_dim = max_x - min_x
        y_dim = max_y - min_y

        patch_across_stains = []
        coord_across_stains = []

        patch_across_stains.append(image1[min_y:max_y, min_x:max_x])
        coord_across_stains.append(coords[i])

        coords_norm2 = normalize_coords(image1, image2, coords[i])
        coord_across_stains.append(coords_norm2)

        xmin_norm2, ymin_norm2, xmax_norm2, ymax_norm2 = coords_norm2
        patch2_norm = image2[ymin_norm2:ymax_norm2, xmin_norm2:xmax_norm2]
        patch_across_stains.append(cv2.resize(patch2_norm, (x_dim, y_dim)))

        coords_norm3 = normalize_coords(image1, image3, coords[i])
        coord_across_stains.append(coords_norm3)

        xmin_norm3, ymin_norm3, xmax_norm3, ymax_norm3 = coords_norm3
        patch3_norm = image3[ymin_norm3:ymax_norm3, xmin_norm3:xmax_norm3]
        patch_across_stains.append(cv2.resize(patch3_norm, (x_dim, y_dim)))

        # Check if the patch contains parts from all three images
        if all(np.any(patch > 0) for patch in patch_across_stains):
            # Check if the patch has between 5-80% black pixels - this is approximately the range we want
            black_pixels = [np.sum(np.all(patch == [0, 0, 0], axis=-1)) for patch in patch_across_stains]
            total_pixels = patch_across_stains[0].shape[0] * patch_across_stains[0].shape[1]
            black_pixel_percentages = [black_pixel / total_pixels for black_pixel in black_pixels]

            if all(percentage <= 0.8 for percentage in black_pixel_percentages):
                patches_across_stains.append(patch_across_stains)
                coords_across_stains.append(coord_across_stains)
    print("Patches that passed filtering:", len(patches_across_stains))

    return patches_across_stains, coords_across_stains

## Mask Alignment Using Affine Transformations

In [ ]:
def align_masks(fixed_mask, moving_mask, warp_mode=cv2.MOTION_AFFINE, number_of_iterations=5000, termination_eps=1e-10):
    fixed_gray = cv2.cvtColor(fixed_mask, cv2.COLOR_BGR2GRAY) if len(fixed_mask.shape) == 3 else fixed_mask
    moving_gray = cv2.cvtColor(moving_mask, cv2.COLOR_BGR2GRAY) if len(moving_mask.shape) == 3 else moving_mask

    fixed_gray = fixed_gray.astype(np.float32)
    moving_gray = moving_gray.astype(np.float32)

    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else:
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

    cc, warp_matrix = cv2.findTransformECC(fixed_gray, moving_gray, warp_matrix, warp_mode, criteria)

    sz = fixed_mask.shape
    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        aligned_mask = cv2.warpPerspective(moving_mask, warp_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else:
        aligned_mask = cv2.warpAffine(moving_mask, warp_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

    return aligned_mask

def align_all_masks(he_mask, melan_mask, sox10_mask):
    aligned_melan = align_masks(he_mask, melan_mask)
    aligned_sox10 = align_masks(he_mask, sox10_mask)
    return he_mask, aligned_melan, aligned_sox10

## Patch Export Pipeline

In [ ]:
image_folder = "testing_slices" # put folder path here
output_path = 'testing_slices/patch_outputs'

# Dictionary to store matches
cases = {}

# Iterate over files in the folder
for filename in os.listdir(image_folder):
    match = re.match(r"case_(\d+)_match_(\d+)_(h&e|melan|sox10).*\.(png|tif)", filename)
    if match:
        case_number, match_number, scan_type, extension = match.groups()
        case_number, match_number = int(case_number), int(match_number)  # Convert to int for sorting

        if case_number not in cases:
            cases[case_number] = {}

        if match_number not in cases[case_number]:
            cases[case_number][match_number] = {}

        if scan_type not in cases[case_number][match_number]:
            cases[case_number][match_number][scan_type] = {"mask": "None", "slice": "None"}

        if extension == "png":
            cases[case_number][match_number][scan_type]["mask"] = os.path.join(image_folder, filename)

        elif extension == "tif":
            cases[case_number][match_number][scan_type]["slice"] = os.path.join(image_folder, filename)

# Build list of full triplets
matched_triplets = []
for case in cases.values():
    for match in case.values():
        if all(stain in match for stain in ['h&e', 'melan', 'sox10']):
            h_mask = match['h&e'].get('mask')
            h_slice = match['h&e'].get('slice')
            m_mask = match['melan'].get('mask')
            m_slice = match['melan'].get('slice')
            s_mask = match['sox10'].get('mask')
            s_slice = match['sox10'].get('slice')

            if all([h_mask, h_slice, m_mask, m_slice, s_mask, s_slice]):
                matched_triplets.append((h_mask, h_slice, m_mask, m_slice, s_mask, s_slice))

# delete any patches that are already in the directory
empty_folder()

# Run patching and exporting over triplets
for i, triplet in enumerate(matched_triplets, 1):
    h_mask, h_slice, m_mask, m_slice, s_mask, s_slice = triplet

    # Use H&E mask to generate patch coordinates
    image1 = cv2.imread(h_mask)
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    Otsu_threshold, epithelium_mask_2D = cv2.threshold(image1_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    case_number = re.search(r"case_(\d+)_match_(\d+)", h_mask).groups()
    case_number = tuple(map(int, case_number))

    # Read original RGB slice images
    he_img = cv2.cvtColor(cv2.imread(h_slice), cv2.COLOR_BGR2RGB)
    melan_img = cv2.cvtColor(cv2.imread(m_slice), cv2.COLOR_BGR2RGB)
    sox10_img = cv2.cvtColor(cv2.imread(s_slice), cv2.COLOR_BGR2RGB)

    # Run patching pipeline
    triplet_patches, patch_length = patching_export(
        epithelium_mask_2D,
        h_mask,
        h_slice,
        [he_img, melan_img, sox10_img]
    )

    # Save output patches
    file_name = os.path.basename(h_mask).replace('mask', '').replace('.png', '')
    for j, triplet in enumerate(triplet_patches):
        for k, stain_patch in enumerate(triplet):
            stain_name = ["h&e", "melan", "sox10"][k]
            output_path = os.path.join("testing_slices", "patch_outputs", f"{file_name}_patch{j}_{stain_name}.png")
            cv2.imwrite(output_path, stain_patch)
